In [100]:
import os
from os import walk
from bs4 import BeautifulSoup
import re
import pandas as pd

In [101]:
dir_path = os.path.join('.','AWR')
awr_paths = []
for (dir_path, dir_names, file_names) in walk(dir_path):
    file_paths = [os.path.join(dir_path, file_name) for file_name in file_names]
    awr_paths.extend(file_paths)

In [103]:

output = []
for awr_path in awr_paths:
    print(awr_path)
    with open(awr_path) as f:
        soup = BeautifulSoup(f)

    row = {}
    row['AWR file'] = awr_path

    # find elapsed time
    elapsed_time = soup.find('td', string=re.compile('Elapsed:')).parent.find('td', string=re.compile('mins')).string.strip()
    elapsed_time = re.compile('\d*\.\d*').search(elapsed_time).group(0)
    row['elapsed_time'] = elapsed_time

    # find db time
    db_time = soup.find('td', string=re.compile('DB Time:')).parent.find('td', string=re.compile('mins')).string.strip()
    db_time = re.compile('\d*\.\d*').search(db_time).group(0)
    row['db_time'] = db_time

    output.append(row)

    break

output = pd.DataFrame(output)
print(output)
    

.\AWR\awr_report_ADTGT_10-11.html
                            AWR file elapsed_time db_time
0  .\AWR\awr_report_ADTGT_10-11.html       120.16  113.75
